In [2]:
import pandas as pd
from src.search.candidate_generator import CandidateGenerator
from src.search.contextual_bandit import ContextualBanditReranker
from src.utils.target_metrics import calculate_delta_ndcg_rewards
from src.search.cross_encoder_reranker import CrossEncoderReranker
from src.utils.target_metrics import get_serp_clicks, ndcg
from src.utils.items_data import ItemsData
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ddsharafiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ddsharafiev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/ddsharafiev/innopolis_projects/rl4search/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ddsharafiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ddsharafiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ddsharafiev/nltk_data...
[nltk_data]   Package punkt_tab is already up-t

True

In [3]:
data = pd.read_csv('data/csv/projects.csv')

items_data = ItemsData(df=data)
cg = CandidateGenerator(alpha=0.95, items_data=items_data)

users = pd.read_csv('data/csv/users.csv')

clicks = pd.read_csv('data/csv/clicks.csv')

Preprocessing done.
BM25 index built.
Embedding model loaded.
Embeddings generated.
FAISS index created.
CandidateGenerator initialized.


In [4]:
query = "some recommender project."

candidates = cg.generate_candidates(query, top_k=50)

cb = ContextualBanditReranker(candidates=candidates, items_data=items_data)
cb.rerank(query)
cb_candidates = cb.get_page(0, 3)

cer = CrossEncoderReranker(candidates=candidates, items_data=items_data, batch_size=16)
cer.rerank(query, top_k=50)
ce_candidates = cer.get_page(0, 3)

serp_clicks = get_serp_clicks(1, clicks, candidates)
ndcg_score = ndcg(serp_clicks, k=len(candidates))
print(f"NDCG score for the SERP: {ndcg_score:.4f}")

serp_clicks = get_serp_clicks(1, clicks, cb_candidates)
ndcg_score = ndcg(serp_clicks, k=len(cb_candidates))
print(f"NDCG score for the Contextual Bandit Reranker: {ndcg_score:.4f}")

serp_clicks = get_serp_clicks(1, clicks, ce_candidates)
ndcg_score = ndcg(serp_clicks, k=len(ce_candidates))
print(f"NDCG score for the Cross-Encoder Reranker: {ndcg_score:.4f}")

NDCG score for the SERP: 0.6611
NDCG score for the Contextual Bandit Reranker: 0.6309
NDCG score for the Cross-Encoder Reranker: 1.0000


In [5]:
A, b = cb.A, cb.b

In [ ]:
for epoch in range(100):
    print(f"Epoch {epoch+1}/100")
    for user_id in range(1, 5):
        query = users.loc[users['id'] == user_id, 'short_profile'].values[0]
        candidates = cg.generate_candidates(query, top_k=100)

        serp_clicks = get_serp_clicks(user_id, clicks, candidates)
        baseline_ndcg_score = ndcg(serp_clicks, k=len(candidates))

        cb = ContextualBanditReranker(candidates=candidates, items_data=items_data, A=A, b=b, regularization=0.1, small_sigma=0.05)
        cb.rerank(query)
        displayed_items = cb.get_page(0, 100)

        serp_clicks = get_serp_clicks(user_id, clicks, displayed_items)
        ndcg_score = ndcg(serp_clicks, k=len(displayed_items))

        updates = calculate_delta_ndcg_rewards(
            displayed_items=displayed_items,
            user_clicks_df=clicks,
            user_id=user_id,
            baseline_candidates=candidates, # Передаем весь список кандидатов от CG
            scale_factor=10.0
        )

        cb.update_batch(query, updates)

        print(f"My NDCG: {ndcg_score}, Baseline NDCG: {baseline_ndcg_score}")

Epoch 1/100
My NDCG: 0.5219053331261584, Baseline NDCG: 0.8935229912282503
My NDCG: 0.43342759516575724, Baseline NDCG: 0.9548985973738172
My NDCG: 0.4410508080708755, Baseline NDCG: 0.7688623902132022
My NDCG: 0.8968188297558539, Baseline NDCG: 0.9715144671217378
Epoch 2/100
My NDCG: 0.5226562000911484, Baseline NDCG: 0.8935229912282503
My NDCG: 0.43343734260053796, Baseline NDCG: 0.9548985973738172
My NDCG: 0.44261227258502633, Baseline NDCG: 0.7688623902132022
My NDCG: 0.8969721100692893, Baseline NDCG: 0.9715144671217378
Epoch 3/100
My NDCG: 0.522029832179199, Baseline NDCG: 0.8935229912282503
My NDCG: 0.43344995934735775, Baseline NDCG: 0.9548985973738172
My NDCG: 0.4419880779846228, Baseline NDCG: 0.7688623902132022
